In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, Float
from config import db_name, user, password, host
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from geoalchemy2 import Geometry
import geopandas as gpd
import pandas as pd
from shapely import wkt
import psycopg2

In [ ]:
# import packages
import requests
import json
import pandas as pd
import geopandas as gpd

In [ ]:
pip install GeoAlchemy2 SQLAlchemy psycopg2

In [ ]:
# send the request
def jason2dataf(url):
    response = requests.get(url)
    response
    # store the raw text of the response in a variable
    raw_data = response.text
    raw_data
    # parse the raw text response into a JSON
    data = json.loads(raw_data)
    data
    # from JSON to Pandas DataFrame
    data_df = pd.json_normalize(data)
    return data_df

In [ ]:
#dataf_liguria = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/regioni/7')
dataf_lombardia = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/regioni/3')
#dataf_Piemonte = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/regioni/1')
#dataf_valle = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/regioni/2')
data_prov_milan = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/15')
data_prov_varese = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/12')
data_prov_como = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/13')
data_prov_lecco = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/97')
data_prov_sondrio = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/14')
data_prov_bergamo = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/16')
data_prov_brescia = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/17')
data_prov_pavia = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/18')
data_prov_cremona = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/19')
data_prov_mantova = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/20')
data_prov_lodi = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/98')
data_prov_Monza = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/108')



In [ ]:
# Merge the DataFrames
merged_df = pd.concat([data_prov_milan, data_prov_varese,data_prov_como,data_prov_lecco,data_prov_sondrio,data_prov_bergamo,data_prov_brescia,data_prov_pavia,data_prov_cremona,data_prov_mantova,data_prov_lodi,data_prov_Monza]).reset_index(drop=True)

# Display the merged DataFrame
merged_df.rename(columns={'cod_prov':'COD_PROV'}, inplace=True)


In [ ]:
shapefile_path_region = "./Limiti01012024_g/Reg01012024_g/Reg01012024_g_WGS84.shp"
shapefile_path_province = "./Limiti01012024_g/ProvCM01012024_g/ProvCM01012024_g_WGS84.shp"
gdf_region = gpd.read_file(shapefile_path_region)
gdf_province = gpd.read_file(shapefile_path_province)

In [ ]:

desired_cod_RIP = [15,12,13,97,14,16,17,18,19,20,98,108]

# Filter the GeoDataFrame based on the desired cod_RIP values
filtered_gdf = gdf_province[gdf_province['COD_PROV'].isin(desired_cod_RIP)]

# Display the filtered DataFrame


In [ ]:
desired_cod_REG = [3]

# Filter the GeoDataFrame based on the desired cod_RIP values
filtered_gdf2 = filtered_gdf[filtered_gdf['COD_RIP'].isin(desired_cod_REG)]

# Display the filtered DataFrame
print(filtered_gdf2)

In [ ]:
merged_df2 = merged_df.merge(filtered_gdf[['COD_PROV', 'geometry']], on='COD_PROV', how='left')

In [ ]:
columns_to_select = ['geometry','cod_reg','cod_rip','COD_PROV','ar_kmq','nome','uid',
    'ar_fr_p3p4', 'ar_fr_p2', 'ar_fr_p1', 
    'pop_fr_p2', 'pop_fr_p1', 
    'ed_fr_p2', 'ed_fr_p1', 
    'ar_frp3p4p', 'popfrp3p4p', 
    'ed_fr_p3p4', 'edfrp3p4p'
]

# Select the columns
selected_df = merged_df2[columns_to_select]
selected_df2 = selected_df.copy()
type(selected_df2['geometry'].iloc[0])

In [ ]:

# Database connection parameters
# with open('config.py', 'w') as f:
#     f.write(db_name = input(f'Enter the database name for the postgres:\n'))
#     f.write(user = input(f'Enter the user for the postgres:\n'))
#     f.write(password = input(f'Enter the Password for the postgres:\n'))
#     f.write(host = input(f'Enter the host for the postgres:\n'))

# db_name = "project"
# user = "postgres"
# password = "admin"
# host = "localhost"
print (db_name, user, password, host)
# db_name = "project"
# user = "postgres"
# password = "admin"
# host = "localhost"

# Connect to the default database to create the new database
conn = psycopg2.connect(dbname="postgres", user=user, password=password, host=host)
conn.autocommit = True # needed to create a db programmatically
cursor = conn.cursor()

# Create the database
try:
    cursor.execute(f"CREATE DATABASE {db_name};")
except: pass

conn.close()

In [ ]:
from shapely.wkb import loads as wkb_loads
from shapely.wkt import dumps as wkt_dumps
from shapely.geometry import MultiPolygon, Polygon
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Text, create_engine
from sqlalchemy.orm import sessionmaker

# Define the Base class for SQLAlchemy
Base = declarative_base()

# Define your SQLAlchemy model
class Dataset(Base):
    __tablename__ = 'dataset'

    id = Column(Integer, primary_key=True)
    cod_reg = Column(Integer)
    cod_rip = Column(Integer)
    cod_prov = Column(Integer)
    ar_kmq = Column(Integer)
    nome = Column(String)
    uid = Column(Integer)
    ar_fr_p3p4 = Column(Integer)
    ar_fr_p2 = Column(Integer)
    ar_fr_p1 = Column(Integer)
    pop_fr_p2 = Column(Integer)
    pop_fr_p1 = Column(Integer)
    ed_fr_p2 = Column(Integer)
    ed_fr_p1 = Column(Integer)
    ar_frp3p4p = Column(Integer)
    popfrp3p4p = Column(Integer)
    ed_fr_p3p4 = Column(Integer)
    edfrp3p4p = Column(Integer)
    geometry = Column(Text)  # Assuming you're storing geometry as WKT in a text column

    def __repr__(self):
        return f"<Dataset(id={self.id}, nome='{self.nome}')>"

# Function to convert WKB to WKT
def convert_wkb_to_wkt(geom):
    if isinstance(geom, str):
        geom = bytes.fromhex(geom)  # Convert hex string to bytes
    if isinstance(geom, (bytes, bytearray)):
        return wkt_dumps(wkb_loads(geom))  # Convert WKB to Shapely geometry, then to WKT
    elif isinstance(geom, (MultiPolygon, Polygon)):
        return wkt_dumps(geom)  # If it's already a Shapely geometry, convert directly to WKT
    else:
        raise TypeError(f"Unexpected geometry type: {type(geom)}")

# Apply conversion function to the 'geometry' column
selected_df2['geometry_wkt'] = selected_df2['geometry'].apply(convert_wkb_to_wkt)

# Print the type of the first geometry to confirm conversion
print(type(selected_df2['geometry_wkt'].iloc[0]))  # Should be <class 'str'>
print(selected_df2['geometry_wkt'].head())  # Optionally, check the first few rows

# Connect to the database (replace with your actual database URI)
# Create an engine and connect to the PostgreSQL database
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:5432/{db_name}')

# Create the table
Base.metadata.create_all(engine)

# Inserting Data into the Database
Session = sessionmaker(bind=engine)
session = Session()

for index, row in selected_df2.iterrows():
    dataset_entry = Dataset(
        cod_reg=row['cod_reg'],
        cod_rip=row['cod_rip'],
        cod_prov=row['COD_PROV'],
        ar_kmq=row['ar_kmq'],
        nome=row['nome'],
        uid=row['uid'],
        ar_fr_p3p4=row['ar_fr_p3p4'],
        ar_fr_p2=row['ar_fr_p2'],
        ar_fr_p1=row['ar_fr_p1'],
        pop_fr_p2=row['pop_fr_p2'],
        pop_fr_p1=row['pop_fr_p1'],
        ed_fr_p2=row['ed_fr_p2'],
        ed_fr_p1=row['ed_fr_p1'],
        ar_frp3p4p=row['ar_frp3p4p'],
        popfrp3p4p=row['popfrp3p4p'],
        ed_fr_p3p4=row['ed_fr_p3p4'],
        edfrp3p4p=row['edfrp3p4p'],
        geometry=row['geometry_wkt']  # Insert WKT geometry
    )
    session.add(dataset_entry)

try:
    session.commit()
except Exception as e:
    session.rollback()
    print(f"Error committing session: {e}")
finally:
    session.close()